In [1]:
'''
Date: 2022.09.06
Title: 
By: Kang Jin Seong
'''

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)


In [2]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

print(train_input.shape, sub_input.shape, val_input.shape)

(5197, 3) (4157, 3) (1040, 3)


In [3]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [4]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0094142 , 0.00746822, 0.00805044, 0.00745273, 0.00700951]), 'score_time': array([0.00096178, 0.00082111, 0.00084043, 0.00072265, 0.00068903]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [5]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [7]:
from sklearn.model_selection import StratifiedKFold
spliter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = spliter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [8]:
# 그리드 서치(하이퍼파라미터와 교차검증을 동시에 수행)

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001,0.0002,0.0003,0.0004,0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = 1)

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [9]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [11]:
'''
Date: 2022.09.07
Title: 
By: Kang Jin Seong
'''

print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [12]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [13]:
# argmax() 함수를 사용하여 가장 큰 값의 인덱스를 추출할 수 있다.
# 인덱스를 사용하여 params 키에 저장된 매개변수를 출력할 수 있다.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [19]:
# 복잡한 매개변수 조합
# min_impurity_decrease : 불순도 감소 최소량
# max_depth: 트리의 깊이
# min_samples_split: 노드를 나누기 위한 최소 샘플 수
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)}

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state =42), params, n_jobs = -1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
# 최상위 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [24]:
# 랜덤 서치
'''
확률 분포 객체: uniform, randint 주어진 범위에서 고르게 값을 뽑습니다.(균등 분포에서 샘플링한다.)
uniform: 실수
randint: 정수
'''
from scipy.stats import uniform, randint
# 연습
rgen = randint(0, 10)
rgen.rvs(10)

array([6, 6, 6, 7, 5, 3, 6, 0, 4, 4])

In [25]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([104, 102, 111,  96, 102,  77,  94,  91, 114, 109]))

In [26]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.08354073, 0.83856981, 0.53636356, 0.27910035, 0.78802086,
       0.03972022, 0.4725768 , 0.02189079, 0.4045655 , 0.96215367])

In [29]:
# min_samples_leaf: 리프노드가 되기 위한 최소 샘플의 개수 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않습니다.
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,52),
          'min_samples_leaf': randint(1,25)}

In [30]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs= -1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f98192518d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9819251710>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9819251c50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9819251a10>},
                   random_state=42)

In [31]:
print(gs.best_params_)

{'max_depth': 28, 'min_impurity_decrease': 0.000404781258158029, 'min_samples_leaf': 6, 'min_samples_split': 9}


In [33]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8693486710594508


In [34]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8576923076923076
